# Wasserbau 4, Uebung 01
## Aufgabe 1 – Hochwasserstatistik mit Python
Dieses Jupyter Notebook behandelt die Analyse von Hochwasserstatistiken mit der Pearson-Typ-III-Verteilung, ermöglicht empirische Häufigkeitsanalysen und beinhaltet die Bemessung eines Hochwasserrückhaltebeckens.

## 1. Pearson-Typ-III-Verteilung – Grundlagen und Berechnung

In [1]:
# Beispielhafte Daten (Scheitelabflüsse in m³/s)
Q = [38.5, 38.1, 35.2, 32.0, 29.5, 29.0, 28.7, 28.2, 25.2, 23.7, 23.3, 22.6, 21.5]

In [2]:
import math
# Berechnung arithmetisches Mittel, Standardabweichung, Schiefe
n = len(Q)
mean_x = sum(Q) / n
s_x = math.sqrt(sum((x - mean_x)**2 for x in Q) / (n - 1))
C_sx = (n * sum((x - mean_x)**3 for x in Q)) / ((n - 1) * (n - 2) * s_x**3)
print(f"Arithmetisches Mittel = {mean_x:.2f} m³/s")
print(f"Standardabweichung = {s_x:.2f} m³/s")
print(f"Schiefekoeffizient C_sx = {C_sx:.3f}")

Arithmetisches Mittel = 28.88 m³/s
Standardabweichung = 5.73 m³/s
Schiefekoeffizient C_sx = 0.492


### Logarithmische Transformation und Pearson-Verteilungsparameter

In [3]:
# Logarithmische Transformation
Y = [math.log10(x) for x in Q]
mean_y = sum(Y) / n
s_y = math.sqrt(sum((y - mean_y)**2 for y in Y) / (n - 1))
C_sy = (n * sum((y - mean_y)**3 for y in Y)) / ((n - 1) * (n - 2) * s_y**3)
print(f"Arithmetisches Mittel (log) = {mean_y:.3f}")
print(f"Standardabweichung (log) = {s_y:.3f}")
print(f"Schiefekoeffizient C_sy = {C_sy:.3f}")

Arithmetisches Mittel (log) = 1.453
Standardabweichung (log) = 0.085
Schiefekoeffizient C_sy = 0.231


In [4]:
# k-Wert-Interpolation
def interpolate_k(Cs, Cs1, k1, Cs2, k2):
    return k1 + (k2 - k1) * (Cs - Cs1) / (Cs2 - Cs1)

# Beispiel: Interpolation für T=50a und 100a
k_T50 = interpolate_k(C_sy, 0.2, 2.159, 0.3, 2.211)
k_T100 = interpolate_k(C_sy, 0.2, 2.472, 0.3, 2.544)
y_T50 = mean_y + s_y * k_T50
y_T100 = mean_y + s_y * k_T100
Q_T50 = 10**y_T50
Q_T100 = 10**y_T100
print(f"50-jährliches Hochwasser: {Q_T50:.2f} m³/s")
print(f"100-jährliches Hochwasser: {Q_T100:.2f} m³/s")

50-jährliches Hochwasser: 43.39 m³/s
100-jährliches Hochwasser: 46.17 m³/s


### Interaktives Widget – Bemessungshochwasser

In [5]:
import ipywidgets as widgets
from ipywidgets import interact

def design_flood(T=100):
    k_table = {25: 1.8277, 50: 2.1752, 100: 2.4945, 200: 2.7920, 1000: 3.4219}
    kT = k_table.get(T, 2.4945)
    yT = mean_y + s_y * kT
    QT = 10**yT
    print(f"{T}-jährliches Hochwasser: Q ≈ {QT:.1f} m³/s")

interact(design_flood, T=widgets.Dropdown(options=[25,50,100,200,1000], value=100))

interactive(children=(Dropdown(description='T', index=2, options=(25, 50, 100, 200, 1000), value=100), Output(…

<function __main__.design_flood(T=100)>

## 2. Empirische Häufigkeit und Häufigkeitsanalyse

In [6]:
import pandas as pd
HQ = sorted([12.7, 14.0, 16.0, 18.2, 19.4, 21.5, 22.6, 23.3, 23.7, 25.2,
             28.2, 28.7, 29.0, 29.5, 32.0, 35.2, 38.1, 38.5, 41.7, 43.5,
             43.7, 44.3, 44.5, 46.7, 48.5, 48.6, 52.7, 53.5, 54.9, 56.0,
             57.2, 57.5, 58.4, 64.0, 69.0, 70.7])
n2 = len(HQ)
data = []
for i, q in enumerate(HQ, start=1):
    P_emp = i/(n2+1) * 100
    T_emp = (n2+1) / (n2+1 - i)
    data.append([i, q, P_emp, T_emp])
df = pd.DataFrame(data, columns=["Rang", "HQ (m³/s)", "P_emp (%)", "T_emp (Jahre)"])
df.head()

,Rang,HQ (m³/s),P_emp (%),T_emp (Jahre)
0,1,12.7,2.702703,1.027778
1,2,14.0,5.405405,1.057143
2,3,16.0,8.108108,1.088235
3,4,18.2,10.810811,1.121212
4,5,19.4,13.513514,1.156250


## 3. Hochwasserrückhaltebecken – Wehrberechnung

In [7]:
def weir_outflow(Q=55.0, b=20.0, mu=0.74):
    c = 1.0
    g = 9.81
    h_Ü = ((3 * Q) / (2 * mu * c * b * math.sqrt(2 * g))) ** (2/3)
    stauziel = 45.00
    freibord = 0.70
    kronenhöhe = stauziel + h_Ü + freibord
    print(f"Überfallhöhe hÜ = {h_Ü:.2f} m")
    print(f"Kronenhöhe (inkl. Freibord) = {kronenhöhe:.2f} m ü. NN")

interact(weir_outflow, Q=(30,80,1), b=(10,40,1), mu=(0.50,0.80,0.01))

interactive(children=(IntSlider(value=55, description='Q', max=80, min=30), IntSlider(value=20, description='b…

<function __main__.weir_outflow(Q=55.0, b=20.0, mu=0.74)>

## 4. Hochwasserentlastung über raue Rampe

In [8]:
def ramp_design(Q=55.0, b=20.0, I_R=0.075, rho_s=2650.0):
    rho_w = 1000.0
    g = 9.81
    q_ist = Q / b
    denominator = 0.8 * 0.257 * math.sqrt((rho_s - rho_w) / rho_w) * math.sqrt(g) * (I_R ** (-7/6))
    d65 = (q_ist / denominator) ** (2/3)
    d_s = 1.06 * d65
    m_s = rho_s * math.pi * (d_s**3) / 6
    print(f"d_65 ≈ {d65:.3f} m, d_s ≈ {d_s:.3f} m, Steinmasse ≈ {m_s:.1f} kg")

interact(ramp_design, Q=(30,80,1), b=(10,40,1), I_R=(0.01,0.15,0.005), rho_s=(2000,3000,50))

interactive(children=(IntSlider(value=55, description='Q', max=80, min=30), IntSlider(value=20, description='b…

<function __main__.ramp_design(Q=55.0, b=20.0, I_R=0.075, rho_s=2650.0)>

In [9]:
# Konstanten
b_klappe = 7.2  # Breite pro Klappe [m]
g = 9.81        # Erdbeschleunigung [m/s²]


In [10]:
import ipywidgets as widgets
from IPython.display import display

# Interaktive Eingabe der Parameter
mu_widget = widgets.FloatSlider(value=0.65, min=0.4, max=1.0, step=0.01, description='μ:')
n_widget = widgets.IntSlider(value=4, min=1, max=5, step=1, description='Klappen n:')
Q_widget = widgets.FloatSlider(value=100, min=10, max=500, step=10, description='Q [m³/s]:')
h1_widget = widgets.FloatSlider(value=1.2, min=0.5, max=3.0, step=0.05, description='h1 [m]:')

display(mu_widget, n_widget, Q_widget, h1_widget)


FloatSlider(value=0.65, description='μ:', max=1.0, min=0.4, step=0.01)

IntSlider(value=4, description='Klappen n:', max=5, min=1)

FloatSlider(value=100.0, description='Q [m³/s]:', max=500.0, min=10.0, step=10.0)

FloatSlider(value=1.2, description='h1 [m]:', max=3.0, min=0.5, step=0.05)

In [11]:
from ipywidgets import interact

def berechne_interaktiv(mu, n, Q, h1):
    A = b_klappe * n
    v1 = Q / (A * h1)
    Fr = v1 / (g * h1)**0.5
    h2_erf = (h1 / 2) * ((8 * Fr**2 + 1)**0.5 - 1)
    print(f"Froude-Zahl Fr = {Fr:.2f}")
    print(f"Erforderliche Tiefe h2_erf = {h2_erf:.2f} m")
    
interact(berechne_interaktiv, mu=mu_widget, n=n_widget, Q=Q_widget, h1=h1_widget)


interactive(children=(FloatSlider(value=0.65, description='μ:', max=1.0, min=0.4, step=0.01), IntSlider(value=…

<function __main__.berechne_interaktiv(mu, n, Q, h1)>

In [12]:
import pandas as pd
import matplotlib.pyplot as plt

data = {
    "h2_erf [m]": [1.49, 2.67, 4.09, 5.76],
    "n=1": [6, 20, 50, 110],
    "n=2": [12, 40, 100, 220],
    "n=3": [18, 60, 150, 330],
    "n=4": [24, 80, 200, 440],
    "n=5": [30, 100, 250, 550],
}
df = pd.DataFrame(data)

klappen_dropdown = widgets.Dropdown(
    options=[1, 2, 3, 4, 5],
    value=4,
    description='Klappenanzahl:',
)

def zeige_diagramm(n_klappen):
    plt.plot(df["h2_erf [m]"], df[f"n={n_klappen}"], marker='o')
    plt.title(f"Wehrbedienungsplan für n={n_klappen} Klappen")
    plt.xlabel("h2_erf [m]")
    plt.ylabel("Abfluss Q [m³/s]")
    plt.grid(True)
    plt.show()

interact(zeige_diagramm, n_klappen=klappen_dropdown)


interactive(children=(Dropdown(description='Klappenanzahl:', index=3, options=(1, 2, 3, 4, 5), value=4), Outpu…

<function __main__.zeige_diagramm(n_klappen)>

### ✍ Aufgabe:
Variieren Sie den Abfluss Q und beobachten Sie, wie sich die erforderliche Wassertiefe h2_erf verändert.
Was passiert bei hoher Froude-Zahl? Wie verändert sich der Wehrbedienungsplan bei geringer Klappenzahl?